In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Circle
import matplotlib.patches as mpatches
import seaborn as sns

plt.rcParams["font.family"] = "Cambria"

In [2]:
models = ["rf", "mlp", "rfmlp", "borfmlp"]
units = {
    "Turbidity": "FNU", 
    "DO": "mg/L", 
    "fChl": "RFU",
}
colors = {
    "rf": {"face": "#f59e0b", "edge": "#451a03"},
    "mlp": {"face": "#3b82f6", "edge": "#082f49"},
    "rfmlp": {"face": "#f43f5e", "edge": "#4c0519"},
    "borfmlp": {"face": "#10b981", "edge": "#022c22"},
}

In [3]:
metrics = pd.read_csv("../dataset/predictions/metrics.csv")
metrics

,Model,Target,MAE,RMSE,R²,MBE
0,borfmlp_fChl,fChl,1.46,1.93,82.58,-0.17
1,rfmlp_fChl,fChl,1.46,1.94,82.56,-0.23
2,rf_fChl,fChl,1.60,2.09,79.61,0.01
3,mlp_fChl,fChl,1.70,2.41,73.05,0.11
4,borfmlp_Turbidity,Turbidity,5.34,7.38,95.61,-0.79
5,rfmlp_Turbidity,Turbidity,5.90,8.11,94.70,-0.87
6,mlp_Turbidity,Turbidity,6.61,9.25,93.09,0.29
7,rf_Turbidity,Turbidity,9.07,12.69,86.99,0.33
8,borfmlp_DO,DO,0.44,0.60,91.41,-0.01
9,rfmlp_DO,DO,0.45,0.61,90.94,0.00


In [4]:
predicts = {
    "Turbidity": pd.read_csv("../dataset/predictions/Turbidity.csv"),
    "DO": pd.read_csv("../dataset/predictions/DO.csv"),
    "fChl": pd.read_csv("../dataset/predictions/fChl.csv"),
}

In [5]:
predicts["Turbidity"].head()

,Turbidity,rf_Turbidity,mlp_Turbidity,borfmlp_Turbidity,rfmlp_Turbidity
0,68.1,71.724,64.458946,66.357403,63.441934
1,43.6,31.436,30.929830,34.996351,31.822760
2,152.0,139.232,147.034587,136.946158,135.914627
3,153.0,105.377,147.662699,137.835245,140.011691
4,22.2,34.359,20.806969,23.506382,24.047535


In [6]:
predicts["DO"].head()

,DO,rf_DO,mlp_DO,borfmlp_DO,rfmlp_DO
0,10.2,10.347,11.652401,10.459597,10.382837
1,10.6,10.104,10.439160,9.834492,9.981621
2,8.6,9.774,7.063467,9.425116,9.500806
3,7.8,9.289,8.349853,8.865154,9.310897
4,13.1,13.105,11.673926,12.922708,12.965227


In [7]:
predicts["fChl"].head()

,fChl,rf_fChl,mlp_fChl,borfmlp_fChl,rfmlp_fChl
0,2.8,3.584,4.746184,3.826191,3.786033
1,6.0,10.480,9.753507,8.598258,8.239133
2,3.2,3.339,2.967564,2.975148,3.354063
3,1.6,3.058,3.320998,2.623374,2.752108
4,2.1,2.171,1.218013,2.192735,2.595071


In [8]:
def metricextract(col, model):
    row = metrics.loc[metrics['Model'] == f"{model}_{col}"]
    mae = row["MAE"].iloc[0]
    rmse = row["RMSE"].iloc[0]
    r2 = row["R²"].iloc[0]
    mbe = row["MBE"].iloc[0]
    return np.around([mae, rmse, r2, mbe], 2)

In [9]:
def scat(ax, i, col, model, limits):
    x = predicts[col][col]
    y = predicts[col][model + "_" + col]
    sns.kdeplot(x=x, y=y, levels=10, cmap='Paired', alpha=0.6, fill=True, ax=ax, zorder=1)
    ax.plot(
        [limits[0], limits[1]],
        [limits[0], limits[1]],
        color='red',
        lw=2,
        alpha=0.5,
        ls='--',
        zorder=2
    )
    ax.scatter([0], [0], label=model.upper(), c='white', s=0)
    ax.scatter(
        x=x, y=y,
        s=180,
        c=colors[model]["face"],
        edgecolors= colors[model]["edge"], lw=2,
        alpha=0.8, 
        zorder=3
    )
    ax.set_xlim(limits)
    ax.set_ylim(limits)
    ax.set_xticks(np.linspace(limits[0], limits[1], 8))
    ax.set_yticks(np.linspace(limits[0], limits[1], 8))
    ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:.0f}"))
    ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:.0f}"))
    
    row, col_idx = divmod(i, 2)
    if col_idx == 1:
        ax.set_yticklabels([])
        ax.set_ylabel("")
    else:
        ax.set_ylabel(
            col + "$_{Pred}$" + " (" + units[col] + ")",
            fontsize=24,
            labelpad=22
        )
    if row == 0:
        ax.set_xticklabels([])
        ax.set_xlabel("")
    else:
        ax.set_xlabel(
            col + "$_{Obs}$" + " (" + units[col] + ")",
            fontsize=24,
            labelpad=22
        )
    ax.tick_params(labelsize=18)
    metrs = metricextract(col, model)
    text = f"MAE: {metrs[0]} ({units[col]})\nRMSE: {metrs[1]} ({units[col]})\nR²: {metrs[2]} (%)\nMBE: {metrs[3]}  ({units[col]})"
    ax.text(
        0.60, 0.05,
        text,
        fontsize=22,
        transform=ax.transAxes
    )
    ax.legend(loc='upper left', fontsize=24, frameon=False, handletextpad=0.0, handlelength=0)

In [10]:
def compreds(col): # Comparison Predictions
    fig, axes = plt.subplots(2, 2, figsize=(18, 18))
    axes = axes.flatten()

    vals = np.concatenate([
        predicts[col][col],
        *[predicts[col][m + "_" + col] for m in models]
    ])
    vmin, vmax = np.nanmin(vals), np.nanmax(vals)
    pad = 0.05 * (vmax - vmin)
    limits = [vmin - pad, vmax + pad]
    
    for i, ax in enumerate(axes):
        scat(ax, i, col, models[i], limits)

    plt.subplots_adjust(wspace=0.08, hspace=0.08)
    
    plt.savefig(
        transparent=False,
        dpi=300,
        format="png",
        fname=f"../plots/results/{col}.png",
        bbox_inches="tight"
    )
    plt.close()

In [11]:
compreds("Turbidity")
compreds("DO")
compreds("fChl")

In [12]:
def style_polar_axis(ax, num_radii):
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_frame_on(False)
    ax.set_xticklabels([])
    ax.set_ylim([0, num_radii + 1])
    ax.set_yticks(np.arange(1, num_radii + 1))
    ax.set_yticklabels([])
    ax.grid(alpha=1, linestyle='--')
    return ax

In [13]:
def add_range_labels(ax, num_radii, values, label):
    range_bbox = {"facecolor": "w", "edgecolor": "none", "pad": 0.1}
    for i in range(num_radii):
        ax.text(
            np.pi, (i + 1) + 0.5, f"{values[i]} {label}", color="black", ha="center", va="center",
            fontsize=16, bbox=range_bbox
        )
    return ax

In [14]:
def circular(ax, values, unit, colors):
    num_radii = len(values)
    ax = style_polar_axis(ax, num_radii)
    for j, val in enumerate(values):
        normalized_val = np.clip(np.abs(val) / 100.0, 0, 1)
        angle = np.linspace(0, normalized_val * 2 * np.pi, num=200)
        radius = np.full_like(angle, j + 1)
        color = colors[j]
        ax.plot(angle, radius, lw=7, color=color, solid_capstyle="butt")
    circle = plt.Circle((0, 0), num_radii + 1, transform=ax.transData._b, color='black', fill=False, linewidth=5)
    ax.add_artist(circle)
    for line in ax.yaxis.get_gridlines():
        line.set_clip_path(circle)
    for line in ax.xaxis.get_gridlines():
        line.set_clip_path(circle)
    ax = add_range_labels(ax, num_radii, values, unit)

In [15]:
def cometrs(col): # Comparison Metrics
    fig, axes = plt.subplots(2, 2, figsize=(18, 18), subplot_kw={'projection': 'polar'})
    axes = axes.flatten()
    labels = ["MAE", "RMSE", "R²", "MBE"]
    metrics = {}
    colours = []
    for model in models:
        metrics[model] = metricextract(col, model)
        colours.append(colors[model]["face"])
    for i, ax in enumerate(axes):
        values = []
        for k, v in metrics.items():
            values.append(v[i])
        if i == 2:
            circular(ax, values, f" (%)", colours)
        else:
            circular(ax, values, f" ({units[col]})", colours)
        ax.text(
            np.pi, 0, labels[i], color="black", ha="center", va="center",
            fontsize=22
        )
    plt.subplots_adjust(wspace=0.08, hspace=0.08)
    legend_handles = [
        mpatches.Patch(color=colors[m]["face"], label=m.upper())
        for m in models
    ]
    fig.legend(
        handles=legend_handles,
        loc='upper center',
        ncol=4,
        fontsize=18,
        frameon=True,
        bbox_to_anchor=(0.5, 0.95)
    )
    plt.savefig(
        transparent=False,
        dpi=300,
        format="png",
        fname=f"../plots/results/metrs-{col}.png",
        bbox_inches="tight"
    )
    plt.close()

In [16]:
cometrs("Turbidity")
cometrs("DO")
cometrs("fChl")